In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
titanic=pd.read_csv('train.csv', index_col='PassengerId')
test_titanic=pd.read_csv('test.csv', index_col='PassengerId')

In [3]:
#Name title
import re
def name_title(name_list):
    title_list=[]
    for name in name_list:
        title_list.append(re.split(r"^([^.]*).*",  re.split(',', name)[1])[1].strip())
    return title_list

name_list=list(titanic.Name)
title_list=name_title(name_list)
titanic['title']=title_list

name_list=list(test_titanic.Name)
title_list=name_title(name_list)
test_titanic['title']=title_list

In [4]:
titanic.Age.fillna(np.round(titanic[titanic['Pclass']==titanic.Pclass].Age.mean()), inplace=True)
titanic.Embarked.fillna('S', inplace=True)

In [5]:
test_titanic.Age.fillna(np.round(test_titanic[test_titanic['Pclass']==test_titanic.Pclass].Age.mean()), inplace=True)
test_titanic.Fare.fillna(np.round(test_titanic[test_titanic['Pclass']==test_titanic.Pclass].Fare.mean()), inplace=True)
test_titanic.Embarked.fillna('S', inplace=True)

In [6]:
#CHANGES1

titanic['relatives']=titanic['SibSp']+titanic['Parch']
titanic['FarePH']=titanic['Fare']/(titanic['relatives']+1)
titanic=titanic.drop(['Ticket', 'Cabin','Name', 'Fare'], axis=1)


In [7]:
#CHANGES2

test_titanic['relatives']=test_titanic['SibSp']+test_titanic['Parch']
test_titanic['FarePH']=test_titanic['Fare']/(test_titanic['relatives']+1)
test_titanic=test_titanic.drop(['Ticket', 'Cabin', 'Name', 'Fare'], axis=1)

In [8]:
titanic.Sex=titanic.Sex.astype('category')
titanic.Embarked =titanic.Embarked.astype('category')
titanic.Pclass =titanic.Pclass.astype('category')
titanic.title =titanic.title.astype('category')

titanic=pd.get_dummies(titanic)
print(titanic.head(5))

             Survived   Age  SibSp  Parch  relatives    FarePH  Pclass_1  \
PassengerId                                                                
1                   0  22.0      1      0          1   3.62500         0   
2                   1  38.0      1      0          1  35.64165         1   
3                   1  26.0      0      0          0   7.92500         0   
4                   1  35.0      1      0          1  26.55000         1   
5                   0  35.0      0      0          0   8.05000         0   

             Pclass_2  Pclass_3  Sex_female  ...  title_Master  title_Miss  \
PassengerId                                  ...                             
1                   0         1           0  ...             0           0   
2                   0         0           1  ...             0           0   
3                   0         1           1  ...             0           1   
4                   0         0           1  ...             0           0   

In [9]:
test_titanic.Sex=test_titanic.Sex.astype('category')
test_titanic.Embarked =test_titanic.Embarked.astype('category')
test_titanic.Pclass =test_titanic.Pclass.astype('category')
test_titanic.title =test_titanic.title.astype('category')

test_titanic=pd.get_dummies(test_titanic)
print(test_titanic.head(5))

              Age  SibSp  Parch  relatives    FarePH  Pclass_1  Pclass_2  \
PassengerId                                                                
892          34.5      0      0          0  7.829200         0         0   
893          47.0      1      0          1  3.500000         0         0   
894          62.0      0      0          0  9.687500         0         1   
895          27.0      0      0          0  8.662500         0         0   
896          22.0      1      1          2  4.095833         0         0   

             Pclass_3  Sex_female  Sex_male  ...  Embarked_S  title_Col  \
PassengerId                                  ...                          
892                 1           0         1  ...           0          0   
893                 1           1         0  ...           1          0   
894                 0           0         1  ...           0          0   
895                 1           0         1  ...           1          0   
896              

In [10]:
titanic=titanic.drop(['Sex_male', 'Embarked_S'], axis=1)
test_titanic=test_titanic.drop(['Sex_male', 'Embarked_S'], axis=1)


In [11]:
#Standard Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

titanic.Age=scaler.fit_transform(np.array(titanic.Age).reshape(-1, 1))
# titanic.Fare=scaler.fit_transform(np.array(titanic.Fare).reshape(-1, 1))
titanic.relatives=scaler.fit_transform(np.array(titanic.relatives).reshape(-1, 1))
titanic.FarePH=scaler.fit_transform(np.array(titanic.FarePH).reshape(-1, 1))


test_titanic.Age=scaler.fit_transform(np.array(test_titanic.Age).reshape(-1, 1))
# test_titanic.Fare=scaler.fit_transform(np.array(test_titanic.Fare).reshape(-1, 1))
test_titanic.relatives=scaler.fit_transform(np.array(test_titanic.relatives).reshape(-1, 1))
test_titanic.FarePH=scaler.fit_transform(np.array(test_titanic.FarePH).reshape(-1, 1))



In [12]:
#For prediction Score
from sklearn.model_selection import train_test_split
X=titanic.drop(['Survived'], axis=1)
y=titanic['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


# #For Kaggle Competition
# X_train=titanic.drop(['Survived'], axis=1)
# y_train=titanic['Survived']
# X_test=test_titanic

In [13]:
PID=X_test.index
results=pd.DataFrame()
results['PassengerId']=PID
# print(results)

In [14]:
#Adding left over columns to Test dataset (Kaggle)
train_col=X_train.columns
test_col=X_test.columns

add_test_title=list(set(train_col) - set(test_col))
for title in add_test_title:
    X_test[title]=0

add_train_title=list(set(test_col) - set(train_col))
for title in add_train_title:
    X_train[title]=0 

# X_train=X_train.drop(drop_columns, axis=1)
# X_test=X_test.drop(drop_columns, axis=1)    

In [15]:
#logistic Regression
from sklearn.linear_model import  LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
lr_pred= logreg.predict(X_test)

# log_results=results
# log_results['Survived']=lr_pred

print("Accuracy:",accuracy_score(y_test, lr_pred))

Accuracy: 0.8268156424581006


C:\Users\Steve\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
# Naive  Bayes Algorithm
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, y_train)
nb_pred = gnb.predict(X_test) 

# NB_results=results
# NB_results['Survived']=nb_pred

print("Accuracy:",accuracy_score(y_test, nb_pred))

Accuracy: 0.6201117318435754


In [17]:
#KNN Algorithm finding best fit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

for i in range(1, 10):
    classifier = KNeighborsClassifier(n_neighbors=i)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print("i value:",i,"    Acc:",accuracy_score(y_test, y_pred))

i value: 1     Acc: 0.770949720670391
i value: 2     Acc: 0.8044692737430168
i value: 3     Acc: 0.8435754189944135
i value: 4     Acc: 0.8268156424581006
i value: 5     Acc: 0.8324022346368715
i value: 6     Acc: 0.8324022346368715
i value: 7     Acc: 0.8156424581005587
i value: 8     Acc: 0.8044692737430168
i value: 9     Acc: 0.8156424581005587


In [18]:
#KNN Algorithm
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)
knn_pred=knn.predict(X_test)

# KNN_results=results
# KNN_results['Survived']=knn_pred

print("Accuracy:",accuracy_score(y_test, knn_pred))

Accuracy: 0.8435754189944135


In [20]:
#Random Forest Algorithm
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 100)

rf.fit(X_train, y_train)
rf_pred= rf.predict(X_test)

# RF_results=results
# RF_results['Survived']=rf_pred

print("Accuracy:",accuracy_score(y_test, rf_pred))

Accuracy: 0.8379888268156425


In [21]:
#Random Forest Tuning

from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}



rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)


print(rf_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.4min finished


{'n_estimators': 2000, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': False}


In [23]:
#Random Forest Algorithm-2

rf = RandomForestClassifier(n_estimators= 2000, min_samples_split= 10,  min_samples_leaf= 2, max_features= 'auto',
                            max_depth= 50, bootstrap= False)
rf.fit(X_train, y_train)
rf_pred2= rf.predict(X_test)

# RF_results2=results
# RF_results2['Survived']=rf_pred2

print("Accuracy:",accuracy_score(y_test, rf_pred2))

Accuracy: 0.8547486033519553


In [24]:
#Feature Extraction
score=[]
for name, importance in zip(X_train.columns, rf.feature_importances_):
    score.append(importance)

score_df=pd.DataFrame(list(zip(X_train.columns, score)), columns=['features', 'importance'])
score_df= score_df.sort_values(by=['importance'], ascending=False)[:17]
feat=list(score_df['features'])

print(score_df)

drop_columns=list(set(X_train)-set(feat))

        features  importance
22      title_Mr    0.188195
8     Sex_female    0.172062
4         FarePH    0.170327
0            Age    0.109130
7       Pclass_3    0.059569
3      relatives    0.052197
19    title_Miss    0.051412
23     title_Mrs    0.049213
5       Pclass_1    0.035644
1          SibSp    0.033821
2          Parch    0.017244
18  title_Master    0.014431
6       Pclass_2    0.014207
10    Embarked_Q    0.013236
9     Embarked_C    0.012064
25     title_Rev    0.004717
14      title_Dr    0.001640


In [25]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred= dt.predict(X_test)

# DT_results=results
# DT_results['Survived']=dt_pred

print("Accuracy:",accuracy_score(y_test, dt_pred))

Accuracy: 0.7821229050279329


In [26]:
#Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_test)

# SGD_results=results
# SGD_results['Survived']=sgd_pred
 
print("Accuracy:",accuracy_score(y_test, sgd_pred))

Accuracy: 0.6983240223463687


In [27]:
#Voting/Stacking Classifier
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[ ('KNN', knn), ('RanFor', rf)], voting='hard')
voting_clf.fit(X_train, y_train)
y_pred= voting_clf.predict(X_test)

# V_results=results
# V_results['Survived']=y_pred

print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.8491620111731844


In [ ]:
# log_results.to_csv('log_results.csv',index=False)
# # NB_results.to_csv('NB_results.csv',index=False)
# KNN_results.to_csv('KNN_results.csv',index=False)
# RF_results2.to_csv('RF_results.csv',index=False)
# # DT_results.to_csv('DT_results.csv',index=False)
# # SGD_results.to_csv('SGD_results.csv',index=False)
# # V_results.to_csv('V_results.csv',index=False)

